In [134]:
#get_data 데이터수신
# 필요파라미터 : 날짜, 유형(시간)
# minutes : 분 단위. 가능한 값 : 1, 3, 5, 10, 15, 30, 60, 240
# days : 일 단위.
# weeks: 주단위
# months : 월단위
# 날짜 포맷 : yyyy-MM-dd HH:mm:ss
# import json
# import requests
# import numpy as np
# from datetime import datetime
# import datetime as dt
def get_coindata(coinname="BTC",getunit="days",timm="",to=""):#to 는 날짜 포맷스트링
    if getunit=="minutes" and not timm:
        timm=60
    conv_date = datetime.now()
    cut_date = datetime.strptime(to,"%Y-%m-%d %H:%M:%S")
    datasets=[]
    if to :
        while cut_date<conv_date:
            # 날짜 포맷 : yyyy-MM-dd HH:mm:ss
            to = conv_date.strftime("%Y-%m-%d %H:%M:%S")
            url = f"https://api.bithumb.com/v1/candles/{('minutes/'+str(timm) if getunit=='minutes' else getunit)}?market=KRW-{coinname}&count=200{'&to='+to if to else ''}"
            headers = {"accept": "application/json"}    
            response = requests.get(url, headers=headers)
            # if(datasets):
            #     print(datetime.strptime(datasets[-1]["candle_date_time_kst"],"%Y-%m-%dT%H:%M:%S"))
            jData = json.loads(response.text)
            if datasets:
                highDate = datetime.strptime(datasets[-1]["candle_date_time_kst"],"%Y-%m-%dT%H:%M:%S")
                lowDate = datetime.strptime(jData[0]["candle_date_time_kst"],"%Y-%m-%dT%H:%M:%S")
                #중복데이터가 있을시 제거
                if highDate<=lowDate:
                    print("중복발생==")
                    for ix in range(len(jData)):
                        lowDate = datetime.strptime(jData[ix]["candle_date_time_kst"],"%Y-%m-%dT%H:%M:%S")
                        if highDate<=lowDate:
                            jData.pop(ix)
                        else:break
                # 배열 순서 확인, 날짜 내림차순으로 정렬
                # print("이전배열 ======")
                # print(datasets[0]["candle_date_time_kst"])
                # print(datasets[-1]["candle_date_time_kst"]) 더 큰값
                # print("이후배열-----")
                # print(jData[0]["candle_date_time_kst"]) 더 작은값
                # print(json.loads(response.text)[-1]["candle_date_time_kst"])
            datasets.extend(jData)
            conv_date = conv_date-dt.timedelta(days=200)
    else :
        url = f"https://api.bithumb.com/v1/candles/{('minutes/'+str(timm) if getunit=='minutes' else getunit)}?market=KRW-{coinname}&count=200"
        headers = {"accept": "application/json"}    
        response = requests.get(url, headers=headers) 
        datasets.extend(json.loads(response.text))
    return datasets#dumps to json (load, dump  파일컨트롤)


In [165]:
def create_rnndata(dataset,timestep=30):
    print(len(dataset))
    dataset.reverse()#오름차순 정렬 변경
    # print(dataset[-1])
    x_data=[];y_data=[]
    for ix in range(len(dataset)-timestep):
        # ["opening_price","high_price","low_price","trade_price","prev_closing_price"]])# x
        # 산점도 분석후 데이터
        slice_data = [[d["opening_price"],d["high_price"],d["low_price"],d["trade_price"],\
                       d["prev_closing_price"]] for d in dataset[ix:timestep+ix]]
        x_data.append(slice_data)
        y_data.append(dataset[timestep+ix]) # y
    # print(y_data[-1])
    # print(x_data[-1][-1])
    y_data = [int((d["opening_price"]+d["trade_price"])/2*10000)/10000 for d in y_data]
    return np.array(x_data),np.array(y_data)
def high_create_rnndata(dataset,timestep=30):
    print(len(dataset))
    dataset.reverse()#오름차순 정렬 변경
    # print(dataset[-1])
    x_data=[];y_data=[]
    for ix in range(len(dataset)-timestep):
        # ["opening_price","high_price","low_price","trade_price","prev_closing_price"]])# x
        # 산점도 분석후 데이터
        slice_data = [[d["high_price"]] for d in dataset[ix:timestep+ix]]
        x_data.append(slice_data)
        y_data.append(dataset[timestep+ix]) # y
    # print(y_data[-1])
    # print(x_data[-1][-1])
    y_data = [d["high_price"] for d in y_data]
    return np.array(x_data),np.array(y_data)
    
#BTC_days_T30 , 어제데이터도 출력,, 어제하고 갭을 출력해서 오늘 확률의 편차
def get_xpred(coinname="BTC",getunit="days",timm="",timestep=30):
    url = f"https://api.bithumb.com/v1/candles/{('minutes/'+str(timm) if getunit=='minutes' else getunit)}?market=KRW-{coinname}&count={timestep+1}"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers) 
    datasets=json.loads(response.text)
    if len(datasets)<timestep+1:
       return np.array([])
    datasets.reverse()
    y_curprice  = int((datasets[-1]["opening_price"]+datasets[-1]["trade_price"])/2*10000)/10000
    slice_data = [[d["opening_price"],d["high_price"],d["low_price"],d["trade_price"],\
                       d["prev_closing_price"]] for d in datasets[-timestep:]]
    x_data=(slice_data)
    slice_data = [[d["opening_price"],d["high_price"],d["low_price"],d["trade_price"],\
                       d["prev_closing_price"]] for d in datasets[-timestep-1:-1]]
    x_predata=(slice_data)
    return np.array(x_data),np.array(x_predata),y_curprice

In [166]:
def anal_scatter(x_data,y_data):
    import matplotlib.pyplot as plt
    print(x_data[0][0])
    #opening_price # high_price #low_price #trade_price #candle_acc_trade_price
    #candle_acc_trade_volume #prev_closing_price #change_price #change_rate
    anal_list = ["opening_price","high_price","low_price","trade_price","candle_acc_trade_price",\
                "candle_acc_trade_volume","prev_closing_price","change_price", "change_rate"]
    plt.figure(figsize=(8,4))
    for i in range(9):
        plt.subplot(2,5,i+1)
        x_anal=[]
        for d in x_data:
            x_anal.append([f[anal_list[i]] for f in d])
        plt.scatter(np.array(x_anal).mean(axis=1),y_data,s=1)
        plt.title(anal_list[i])
    plt.show()
        

In [168]:
if __name__=="__main__":
    rawdata = (get_coindata("btc",to="2016-03-02 23:59:00"))
    x_data,y_data=create_rnndata(rawdata)
    # print(y_data[0])   #trade_price 현재가격(종가)
    #anal_scatter(x_data,y_data)
    #opening_price, high_price, low_price, trade_price,prev_close_price
    # print(x_data[0],y_data[0])
    # print(x_data[1],y_data[1])
    print(x_data.shape,y_data.shape)
    

3400
(3370, 30, 5) (3370,)


In [ ]:

# #datetime
# # ★ 현재 시간 날짜
# current = datetime.datetime.now()#현재 날짜와 시간을 가저옵니다.
# print(current.date())#타임스탬프에서 년월일 추출
# print(current.time())# 타입스탬프에서 시분초 추출 .초는 소숫점 6자리까지 마이크로세컨
# #timedelta 날짜의 차이 또는 시간의 차이값
# #date
# # 날짜생성
# print("---------")
# #★ 날짜 생성
# customdate = datetime.date(2025,1,25)
# #★ 날짜 시간 생성
# aa = datetime.datetime(2025,1,25,10,13,10)
# customdate = datetime.date.fromisoformat("2025-01-25")#문자를 날짜로 변형
# print(customdate.isoformat())#날짜를 문자로 변형
# #★ 날짜 시간 변경
# customdate=customdate.replace(day=26)#날짜 변경
# print(customdate.timetuple())#날짜,시간을 튜플형태로 출력
# #datetime
# #★ 문자를 날짜로 변경
# mydate = datetime.datetime.strptime(f"{'2/29'};1984", "%m/%d;%Y")
# print(mydate)

# print(dt)
# d = datetime.date(2005, 7, 14)
# t = datetime.time(12, 30)
# #★ 날짜 시간 연결
# print(datetime.datetime.combine(d, t))
# #time
# print(datetime.time.fromisoformat('04:23:01'))
# print(datetime.time.fromisoformat('04:23:01.000384'))
# print(current.replace(hour=10))

# #★ 날짜를 문자로 출력
# customdate.strftime("%Y년 %m월 %d일")#y, Y, m , d 날짜서식 지정 // 날짜서식으로 문자를 출력
# datetime.datetime.strptime("21일11월06년 16시30분", "%d일%m월%y년 %H시%M분")# // 문자인 날짜서식을 날짜로 변경
# #밀리세컨 : 1970.1.1 0:0 :0 현재까지 지난 시간을 1/1000 초 나타냅니다.
# #★ 날짜를 밀리세컨 변경
# ts = current.timestamp()
# print(ts)#밀리세컨 변환 단위 : 마이크로세컨
# #★ 밀리세컨을 날짜로 변경
# print(datetime.datetime.fromtimestamp(ts))
# #날짜등의 차이값은 - 
# #현재시간에서 크리스마스 까지 몇일 몇시간 몇분 몇초가 남았는지 계산
# print("***************")
# cur = datetime.datetime.now()
# cris = datetime.datetime(2025,12,25,0,0,0) 
# diff = cris-cur #diff 타입이 timedelta
# print(diff.days)# 두 날짜 차를 일로 환산
# print(diff.seconds)# 두 날짜 차를 초로 환산
# print(diff.microseconds)# 두 날짜 차를 마이크로초로 환산
# print(diff.total_seconds())# 차이날짜의 밀리세컨
# print("==========밀리세컨을 이용한 일시분초")
# gapmilli = cris.timestamp()*1000-cur.timestamp()*1000#밀리세컨 구하기 리턴
# print(gapmilli)
# il = int(gapmilli/1000/60/60/24)
# print(il,"일")
